<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2021.Q1]**
Prof. Alexandre Donizeti Alves

## **Introdução ao spaCy**

### **01. Instalação**

In [1]:
!pip install spacy --upgrade
#!pip install spacy==2.2.3

     |████████████████████████████████| 12.8MB 288kB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
     |████████████████████████████████| 460kB 42.6MB/s 
     |████████████████████████████████| 9.1MB 29.4MB/s 
     |████████████████████████████████| 122kB 42.1MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=3813cbdd3a7527a963f7bb5563fb31bc0316c752b50bd0b065c50feea668fa1c
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: smart-open 4.2.0
    Uninstallin

In [2]:
import spacy
spacy.__version__

'3.0.5'

In [3]:
!python -m spacy download pt

2021-03-29 18:24:27.107577: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     |████████████████████████████████| 22.1MB 1.4MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


### **02. Marcação POS (*Part-Of-Speech*)**

- POS (part-of-speech) atribui para as palavras partes da fala, como substantivos, adjetivos, verbos
- Importante para a detecção de entidades no texto, pois primeiro é necessário saber o que o texto contém
- Lista de tokens: https://spacy.io/api/annotation#pos-tagging
- Português: https://www.sketchengine.eu/portuguese-freeling-part-of-speech-tagset/

In [4]:
pln = spacy.load('pt_core_news_sm')

In [5]:
documento = pln('Estou aprendendo processamento de linguagem natural na UFABC em Santo André')

In [6]:
for token in documento:
  print(token.text, token.pos_)

Estou AUX
aprendendo VERB
processamento NOUN
de ADP
linguagem NOUN
natural ADJ
na ADP
UFABC PROPN
em ADP
Santo PROPN
André PROPN


### **Legenda**

- *lemma*: raiz da palavra
- *pos*: parte da fala (classe gramatical)
- *tag*: informações morfológicas (por exemplo, se o verbo está no passado)
- *dep*: dependência sintática
- *shape*: formato (maiúsculo, minúsculo, dígitos)
- *alpha*: se é alfabético
- *stop*: se é uma *stop word*

In [7]:
for token in documento:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, 
        token.shape_, token.is_alpha, token.is_stop)

Estou Estou AUX AUX aux Xxxxx True True
aprendendo aprender VERB VERB ROOT xxxx True False
processamento processamento NOUN NOUN obj xxxx True False
de de ADP ADP case xx True True
linguagem linguagem NOUN NOUN nmod xxxx True False
natural natural ADJ ADJ amod xxxx True False
na o ADP ADP case xx True True
UFABC UFABC PROPN PROPN obl XXXX True False
em em ADP ADP case xx True True
Santo Santo PROPN PROPN obl Xxxxx True False
André André PROPN PROPN flat:name Xxxxx True False


In [8]:
for token in documento:
  if token.pos_ == 'PROPN':
    print(token.text)

UFABC
Santo
André


### **03. Lematização e Stemização**

- **Lematização**: "Lema" de uma palavra de acordo com seu significado no dicionário - palavra base (análise vocabular e morfológica)
- **Stemização**: Extrair o radical (stem) das palavras

In [9]:
for token in documento:
  print(token.text, token.lemma_)

Estou Estou
aprendendo aprender
processamento processamento
de de
linguagem linguagem
natural natural
na o
UFABC UFABC
em em
Santo Santo
André André


In [10]:
doc = pln('encontrei encontraram encontrarão encontrariam cursando curso cursei')
[token.lemma_ for token in doc]

['encontrar',
 'encontrar',
 'encontrar',
 'encontrar',
 'cursar',
 'cursar',
 'cursar']

### **Comparação stemização (NLTK) x lematização (spaCy)**

In [ ]:
#!pip install nltk --upgrade

In [11]:
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [12]:
stemmer = nltk.stem.RSLPStemmer()
stemmer.stem('aprender')

'aprend'

In [13]:
for token in documento:
  print(token.text, token.lemma_, stemmer.stem(token.text))

Estou Estou est
aprendendo aprender aprend
processamento processamento process
de de de
linguagem linguagem lingu
natural natural natur
na o na
UFABC UFABC ufabc
em em em
Santo Santo sant
André André andré


### **04. Reconhecimento de Entidades Nomeadas**

- NER (*Named Entity Recognition*)
- Encontrar e classificar entidades no texto, dependendo da base de dados que foi usada para o treinamento (pessoa, localização, empresa, numéricos)
- Usado em chatbots para saber o assunto falado
- Siglas: https://spacy.io/api/annotation#named-entities

In [14]:
texto = 'A IBM é uma empresa dos Estados Unidos voltada para a área de informática. Sua sede no Brasil fica em São Paulo e a receita em 2018 foi de aproximadamente 320 bilhões de reais'

In [15]:
documento = pln(texto)

In [16]:
for entidade in documento.ents:
  print(entidade.text, entidade.label_)

IBM ORG
Estados Unidos LOC
Brasil LOC
São Paulo LOC


In [17]:
from spacy import displacy
displacy.render(documento, style = 'ent', jupyter = True)

In [18]:
texto = 'Bill Gates nasceu em Seattle em 28/10/1955 e foi o criador da Microsoft'

In [19]:
documento = pln(texto)
for entidade in documento.ents:
  print(entidade.text, entidade.label_)

Bill Gates PER
Seattle LOC
Microsoft ORG


In [20]:
displacy.render(documento, style = 'ent', jupyter = True)

In [21]:
for entidade in documento.ents:
  if entidade.label_ == 'PER':
    print(entidade.text)

Bill Gates


### **05. *Stop words***

Palavras que aparecem com muita frequência e que não apresentam muito significado (e, a, de, da etc.).

In [22]:
from spacy.lang.pt.stop_words import STOP_WORDS

In [23]:
print(STOP_WORDS)

{'coisa', 'iniciar', 'todos', 'pode', 'estiveste', 'usar', 'dezassete', 'devem', 'quinto', 'mas', 'vossas', 'pelas', 'mil', 'contra', 'se', 'elas', 'à', 'ela', 'nível', 'quieto', 'meus', 'conhecido', 'fez', 'após', 'aqui', 'aqueles', 'vindo', 'obrigado', 'poderá', 'ademais', 'momento', 'eventual', 'estou', 'sim', 'certamente', 'adeus', 'contudo', 'parte', 'conselho', 'dão', 'aquele', 'menos', 'em', 'os', 'tais', 'zero', 'seis', 'corrente', 'cá', 'ambos', 'nada', 'apoia', 'tiveste', 'depois', 'você', 'um', 'sabe', 'nove', 'menor', 'quando', 'vários', 'aí', 'dezoito', 'grandes', 'tivestes', 'lá', 'muito', 'direita', 'ou', 'como', 'antes', 'geral', 'sempre', 'tendes', 'diz', 'faço', 'ser', 'embora', 'estará', 'fui', 'novo', 'tempo', 'meio', 'te', 'deverá', 'tanta', 'fazer', 'vens', 'vossa', 'povo', 'desta', 'desde', 'favor', 'daquela', 'comprida', 'enquanto', 'através', 'segundo', 'também', 'vós', 'meu', 'forma', 'possivelmente', 'grande', 'podem', 'todo', 'de', 'debaixo', 'estado', 'mais

In [24]:
len(STOP_WORDS)

416

In [25]:
pln.vocab['ir'].is_stop

True

In [26]:
pln.vocab['caminhar'].is_stop

False

In [27]:
documento = pln('Estou aprendendo processamento de linguagem natural na UFABC em Santo André')

In [28]:
for token in documento:
  if not pln.vocab[token.text].is_stop:
    print(token.text)

aprendendo
processamento
linguagem
natural
UFABC
Santo
André


### **06. Tokenização**

In [29]:
documento = pln('Estou aprendendo processamento de linguagem natural na UFABC em Santo André')

In [30]:
for token in documento:
  print(token)

Estou
aprendendo
processamento
de
linguagem
natural
na
UFABC
em
Santo
André


In [31]:
documento1 = 'Estou aprendendo processamento de linguagem natural na UFABC em Santo André'
documento1.split(' ')

['Estou',
 'aprendendo',
 'processamento',
 'de',
 'linguagem',
 'natural',
 'na',
 'UFABC',
 'em',
 'Santo',
 'André']

**Referência**:

Notebook disponibilizado no Curso de [Processamento de Linguagem Natural com spaCy e Python](https://iaexpert.academy/courses/processamento-linguagem-natural-spacy-python/).


**Mais informações** em [*Natural Language Processing With spaCy in Python*](https://realpython.com/natural-language-processing-spacy-python/).